In [2]:
!conda env list

# conda environments:
#
base                  *  /home/tonic/miniconda3
CompNet                  /home/tonic/miniconda3/envs/CompNet
labelme                  /home/tonic/miniconda3/envs/labelme



In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from IPython.display import clear_output


In [30]:
p = [[405,  27], [627,  80]]
M = [[-2.32212816e-01,  9.72665003e-01,  1.85585626e+02], [-9.72665003e-01, -2.32212816e-01,  9.17229846e+02]]
m_arr = np.array(M)
p_arr = np.array(p)
p_arr = np.append(p_arr, np.ones([p_arr.shape[0], 1]), axis=1)
print(p_arr.shape)
print(m_arr.shape)
print(np.matmul(p_arr, m_arr.T).round().dtype)

(2, 3)
(2, 3)
float64


In [ ]:
def extractROI(image, valley_points, save_path, name):
    # calculate angle    
    vector = np.array(valley_points[1]) - np.array(valley_points[0])
    unit_vector = vector / np.linalg.norm(vector)
    rad = np.arccos(np.dot(unit_vector, np.array([0,-1])))
    angle = np.rad2deg(rad)
    print("angle: {.2f}".format(angle))
    
    # compute rotation matrix
    h, w, dim = image.shape
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
  
    # compute the new bounding dimensions of the image
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - center[0]
    M[1, 2] += (nH / 2) - center[1]
    
    # rotate image and show result
    rotated_image = cv2.warpAffine(image, M, (nW, nH))
    # cv2.imshow("rotated image", rotated_image)
    
    # Transfrom the point from original to rotated image
    points_arr = np.array(valley_points)
    points_arr = np.append(points_arr, np.ones([points_arr.shape[0], 1]), axis=1)
    rotated_points = np.matmul(points_arr, np.array(M).T)
    print(rotated_points)
    
    # clip image
    # because points selected from left to right, so the second point is upper point
    distance = np.abs(rotated_points[0,1] - rotated_points[1,1])
    up_left_x = rotated_points[1,0] + 0.2*distance
    up_left_y = rotated_points[1,1] - 0.2*distance
    ROI = rotated_image[round(up_left_y):round(up_left_y+1.4*distance) ,round(up_left_x):round(up_left_x+1.4*distance)]
    ROI_resized = cv2.resize(ROI, (224, 224), interpolation=cv2.INTER_AREA)
    
#     cv2.circle(rotated_image, rotated_points[0].round().astype("int32"), 3, (0,0,255), 5, 16)
#     cv2.circle(rotated_image, rotated_points[1].round().astype("int32"), 3, (0,0,255), 5, 16)
#     cv2.rectangle(rotated_image, (round(up_left_x),round(up_left_y)), (round(up_left_x+1.4*distance),round(up_left_y+1.4*distance)), (0,0,255), 5)
#     cv2.imshow("marked rotated image", rotated_image)
#     cv2.imshow("ROI", ROI)
    
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    
    # save image
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    cv2.imwrite(os.path.join(save_path, name), ROI_resized)

In [12]:
def mouse_handler(event, x, y, flags, data):
    if event == cv2.EVENT_LBUTTONDOWN:
        # 標記點位置
        cv2.circle(data['img'], (x,y), 3, (0,0,255), 5, 16) 

        # 改變顯示 window 的內容
        cv2.imshow(data["name"], data['img'])
        
        # 顯示 (x,y) 並儲存到 list中
        print("get points: (x, y) = ({}, {})".format(x, y))
        data['points'].append((x,y))

def get_points(im, name):
    # 建立 data dict, img:存放圖片, points:存放點
    data = {}
    data["name"] = name
    data['img'] = im.copy()
    data['points'] = []
    
    # 建立一個 window
    cv2.namedWindow(name, 0)
    
    # 改變 window 成為適當圖片大小
    h, w, dim = im.shape
    # print("Img height, width: ({}, {})".format(h, w))
    cv2.resizeWindow(name, w, h)
        
    # 顯示圖片在 window 中
    cv2.imshow(name,im)
    
    # 利用滑鼠回傳值，資料皆保存於 data dict中
    cv2.setMouseCallback(name, mouse_handler, data)
    
    # 等待按下任意鍵，藉由 OpenCV 內建函數釋放資源
    flag = 0
    while True:
        k = cv2.waitKey()
        if k == 27:
            # cancel process when get "ESC"
            flag = 0
            break
        if k == 13:
            # finish when get "ENTER", but it can only have 2 points
            if len(data['points']) == 2:
                flag = 1
            else:
                flag = 2
            break
        if k == 122:
            # restart this image when get "Z"
            flag = 2
            break
    
    cv2.destroyAllWindows()
    
    # 回傳點 list
    return (flag, data['points'])

In [13]:
#Read the destination image
img_dst = cv2.imread("./session1_fail/00122.tiff")

print("Click on the screen and press any key for end process")
points  = get_points(img_dst)

print("\npoints list:")
print(points)

Click on the screen and press any key for end process
Img height, width: (600, 800)
get points: (x, y) = (335, 320)
get points: (x, y) = (563, 264)

points list:
(2, [(335, 320), (563, 264)])


In [15]:
import glob, sys

#Read the source image
source_path = "./"
save_path = os.path.join(source_path, "ROIs_possible")
images_path = glob.glob(os.path.join(source_path, "*.tiff"))
ROIs_path = glob.glob(os.path.join(save_path, "*.tiff"))
ROI_names = [path[-10:] for path in ROIs_path]
unmarked_path = [path for path in images_path if path[-10:] not in ROI_names]
print("number of unprocess images: ", len(unmarked_path))
print("Click on the screen and press any key for end process")
print("The key selection MUST start from left to right")

for path in unmarked_path:
    # until finish
    while True:
        image_name = path[-10:]
        print("====={}=====".format(image_name))
        img = cv2.imread(path)
        result  = get_points(img, image_name) # return flag and points

        if result[0] == 0:
            # cancel
            sys.exit()
        if result[0] == 1:
            # finish
            extractROI(img, result[1], save_path, image_name)
            break
        if result[0] == 2:
            # restart
            pass

number of unprocess images:  3
Click on the screen and press any key for end process
The key selection MUST start from left to right
=====01960.tiff=====
get points: (x, y) = (198, 145)
get points: (x, y) = (454, 45)
68.66314070819435
[[206.62630435 613.25755289]
 [206.62630435 338.41941871]]
=====05762.tiff=====
get points: (x, y) = (439, 88)
get points: (x, y) = (657, 141)
103.6646003805569
[[170.7873286  471.68630664]
 [170.7873286  247.33613392]]
=====05444.tiff=====
get points: (x, y) = (389, 40)
get points: (x, y) = (581, 59)
95.65149488149764
[[ 79.84703391 464.05063702]
 [ 79.84703391 271.1128232 ]]


# Reference

https://zhuanlan.zhihu.com/p/143035374

https://kknews.cc/code/3oqxejy.html

https://blog.csdn.net/fanjiule/article/details/81606596

https://blog.csdn.net/yefcion/article/details/79435591